In [ ]:
!pip install -q langchain_community replicate

In [ ]:
import os
from langchain_community.llms import Replicate
from google.colab import userdata

In [ ]:
api_token = userdata.get('REPLICATE_API_TOKEN')
os.environ["REPLICATE_API_TOKEN"] = api_token

model = "ibm-granite/granite-3.3-8b-instruct"
output = Replicate(
model=model,
replicate_api_token=api_token,
)

In [ ]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

filename = 'sentiment_NetizenIndonesianLangugage.csv'

df = pd.read_csv(filename)

print("Dataframe shape:", df.shape)
df.head()

Saving sentiment_NetizenIndonesianLangugage.csv to sentiment_NetizenIndonesianLangugage (1).csv
Dataframe shape: (2194, 3)


,product_name,review_text,sentiment
0,Basic Online Course Learning (BOCL),"Materinya goks abis, auto paham! Instrukturnya...",2.0
1,Premium Online Learning (POL),"Platformnya anjay, interaktif banget, jadi gak...",2.0
2,Private Online Course Learning (POCL),"Mentornya pro player, ngajarnya santuy tapi ng...",2.0
3,Onsite Course Learning (OCL),"Sumpah, ini kursus paling oke! Ilmunya kepake ...",2.0
4,Private Course Onsite Learning (PCOL),Sertifikatnya nampol buat naikin CV. Vibes bel...,2.0


In [ ]:
filename = list(uploaded.keys())[0]

In [ ]:
print(df['sentiment'].value_counts())

sentiment
2.0    1026
0.0     621
1.0     546
Name: count, dtype: int64


In [ ]:
df = df.dropna(subset=['sentiment'])

print("Ukuran data setelah drop NA:", df.shape)


Ukuran data setelah drop NA: (2193, 3)


In [ ]:
df['sentiment'] = df['sentiment'].astype(int)

print(df['sentiment'].value_counts())
print(df['sentiment'].dtype)

sentiment
2    1026
0     621
1     546
Name: count, dtype: int64
int64


Urutan e-course atau e-learning dari sentimen tertinggi

In [ ]:
import pandas as pd

summary = df.groupby('product_name').agg(
    review_count=('sentiment', 'count'),
    sentiment_mean=('sentiment', 'mean')
).reset_index()

summary = summary.sort_values(by='sentiment_mean', ascending=False).reset_index(drop=True)

for idx, row in summary.iterrows():
    print(f"{idx+1}. {row['product_name']}:")
    print(f"   - Average Sentiment: {row['sentiment_mean']:.2f}")
    print(f"   - Number of Reviews: {row['review_count']}\n")


1. Onsite Course Learning (OCL):
   - Average Sentiment: 1.41
   - Number of Reviews: 438

2. Private Online Course Learning (POCL):
   - Average Sentiment: 1.27
   - Number of Reviews: 439

3. Private Course Onsite Learning (PCOL):
   - Average Sentiment: 1.17
   - Number of Reviews: 438

4. Basic Online Course Learning (BOCL):
   - Average Sentiment: 1.07
   - Number of Reviews: 439

5. Premium Online Learning (POL):
   - Average Sentiment: 1.00
   - Number of Reviews: 439



Analisis sentimen berdasarkan kategori

In [ ]:
import pandas as pd

reviews_text = "\n".join(df['review_text'].tolist())

parameters = {
    "top_k": 40,
    "top_p": 0.85,
    "max_tokens": 500,
    "min_tokens": 400,
    "random_seed": 42,
    "repetition_penalty": 1.3,
    "stopping_criteria": "length (500 tokens)",
    "stopping_sequence": None
}


refined_prompt = f"""
Please analyze these user reviews and classify them using the sentiment labels:
0 = Buruk
1 = Netral
2 = Bagus

After classifying, group the results by each sentiment category, and provide a summary for each category as follows:

- Sentimen : [Buruk (analisa dari sentiment 0) | Netral (analisa dari sentiment 1) | Bagus (analisa dari sentiment 2)]
- Categories:
- Content Quality: Provide key points or main issues in simple, concise, and expressive phrases (not full reviews).
- Instructor Performance: Provide key phrases summarizing instructor quality.
- Platform Usability: Provide key phrases summarizing platform ease of use or technical issues.

Make sure the summary phrases are short, clear, and reflect the typical sentiment for each category.

Use the following format exactly:

Sentimen: Buruk (analisa dari sentiment 0)
- Categories:
- Content Quality: [short phrase summarizing content issues]
- Instructor Performance: [short phrase summarizing instructor issues]
- Platform Usability: [short phrase summarizing platform issues]

Sentimen: Netral (analisa dari sentiment 1)
- Categories:
- Content Quality: [...]
- Instructor Performance: [...]
- Platform Usability: [...]

Sentimen: Bagus (analisa dari sentiment 2)
- Categories:
- Content Quality: [...]
- Instructor Performance: [...]
- Platform Usability: [...]

Reviews:
{reviews_text}

Please provide only this grouped summary, with short phrases, no full review texts or extra explanations.
"""

response = output.invoke(refined_prompt, parameters=parameters)

print(response)


Sentimen: Buruk
- Categories:
  - Content Quality: Content often lacks depth, outdated, or not tailored to expectations.
  - Instructor Performance: Instruction often lacks engagement, explanations are poor, or instructors are unprofessional.
  - Platform Usability: Platform may have frequent errors, poor technical support, or be difficult to navigate.

Sentimen: Netral
- Categories:
  - Content Quality: Content is mostly standard, with some modules being excellent and others lacking.
  - Instructor Performance: Instructors are generally okay, but some could provide more detail.
  - Platform Usability: Platform functions well for operational purposes, with minimal technical issues.

Sentimen: Bagus
- Categories:
  - Content Quality: Content is generally good, up-to-date, and well-tailored to specific needs.
  - Instructor Performance: Instructors are engaging, clear, and supportive, adapting to learners' needs.
  - Platform Usability: Platform is user-friendly, stable, and offers flexi